In [1]:
#Clean notebook for data clean up explantions

In [9]:
#imported our Dependancies as needed
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import requests
import json
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

#Importing CSV files found online
state_lockdown_file = pd.read_csv("data_sources/lockdown_us.csv")
state_deaths_cases_file = pd.read_csv("data_sources/liz-friedman-us-covid-19-data-from-nytimes/us-states.csv")
county_population_file = pd.read_csv("data_sources/qventus-covid-19-localized-scenario-planner/qventus-covid-19-localized-scenario-planner/covid_county_population_usafacts.csv")

In [17]:
#URL pulls state data just for the current or last weekday reported
url = "https://api.covidtracking.com/v1/states/current.json"
state_current_data = (requests.get(url)).json()
#Creating dataframe form api request
state_current_data=pd.DataFrame(state_current_data)
#date format is yyyymmdd this line changes to yyyy-mm-dd
state_current_data['date'] = pd.to_datetime(state_current_data['date'], format='%Y%m%d')

state_current_data.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,2020-09-02,AK,6233,370651,NaN,46.0,NaN,NaN,NaN,10.0,...,376884,0,0,0eaef7dbd992985ab0f8d68e7290a358f8b6e44f,0,0,0,0,0,
1,2020-09-02,AL,128239,835994,NaN,959.0,14753.0,NaN,1501.0,NaN,...,964233,17,215,fca39761bbc6be0099b27c6b880230afcfeabc1b,0,0,0,0,0,
2,2020-09-02,AR,62112,675337,NaN,435.0,4341.0,NaN,NaN,90.0,...,737449,27,35,3c318be701fbb9138ced6d0286e3874d5edf35fe,0,0,0,0,0,
3,2020-09-02,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,48ebb399ca7a84fe4e65df7be64d01c2adde9b27,0,0,0,0,0,
4,2020-09-02,AZ,202861,1011066,NaN,752.0,21449.0,248.0,NaN,146.0,...,1213927,21,44,36685dfe649f351d4b5be90630ede6819fded584,0,0,0,0,0,


In [18]:
#URL pulls collection of all states current data from past dates
url = "https://api.covidtracking.com/v1/states/daily.json"
state_daily_data = (requests.get(url)).json()

#Creating dataframe form api request
state_daily_data=pd.DataFrame(state_daily_data)

#date format is yyyymmdd this line changes to yyyy-mm-dd
state_daily_data['date'] = pd.to_datetime(state_daily_data['date'], format='%Y%m%d')

#Dropping US Territories from state list
state_daily_data = state_daily_data.set_index("state")
state_daily_data = state_daily_data.drop(["AS","GU","MP","PR", "VI","DC"])
state_daily_data = state_daily_data.reset_index()

#loop for summing for total state cases
states = state_daily_data["state"].unique()
total_cases_by_state = []

for state in states:
    state_case = state_daily_data.loc[state_daily_data["state"] == state]
    total_cases_by_state.append(state_case["positive"].sum())

state_daily_data.head(300)

,state,date,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,AK,2020-09-02,6233.0,370651.0,NaN,46.0,NaN,NaN,NaN,10.0,...,376884,0,0,0eaef7dbd992985ab0f8d68e7290a358f8b6e44f,0,0,0,0,0,
1,AL,2020-09-02,128239.0,835994.0,NaN,959.0,14753.0,NaN,1501.0,NaN,...,964233,17,215,fca39761bbc6be0099b27c6b880230afcfeabc1b,0,0,0,0,0,
2,AR,2020-09-02,62112.0,675337.0,NaN,435.0,4341.0,NaN,NaN,90.0,...,737449,27,35,3c318be701fbb9138ced6d0286e3874d5edf35fe,0,0,0,0,0,
3,AZ,2020-09-02,202861.0,1011066.0,NaN,752.0,21449.0,248.0,NaN,146.0,...,1213927,21,44,36685dfe649f351d4b5be90630ede6819fded584,0,0,0,0,0,
4,CA,2020-09-02,712052.0,10868318.0,NaN,4851.0,NaN,1339.0,NaN,NaN,...,11580370,145,0,61ce2c9982d886c3da71512bb44db09f2bd735a6,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,VT,2020-08-28,1589.0,125166.0,NaN,11.0,NaN,NaN,NaN,NaN,...,126755,0,0,8313ab333124a5f3f0ecc99c9a77bc1740312f2a,0,0,0,0,0,
296,WA,2020-08-28,72703.0,1335461.0,NaN,347.0,6674.0,NaN,NaN,45.0,...,1408164,10,34,cd3f9e36dac565874adaa79baadbcb639dd10d9e,0,0,0,0,0,
297,WI,2020-08-28,78868.0,1156807.0,116.0,309.0,5736.0,103.0,1010.0,NaN,...,1235675,2,52,6e1991c70ac6f2c858e234d5dcbdaae1515fb964,0,0,0,0,0,
298,WV,2020-08-28,9824.0,409757.0,NaN,133.0,NaN,50.0,NaN,23.0,...,419581,3,0,7cd8be047725729195d2e4d578bc2acd3b2eae50,0,0,0,0,0,


In [10]:
#loop for finding positive rate per month

months = [1,2,3,4,5,6,7,8]
positive_rates = []
state_list = []
month_list = []

for state in states:
    
    single_state = state_daily_data.loc[state_daily_data["state"] == state]
    single_state['date'] = pd.to_datetime(single_state['date'], format='%Y%m%d')    
    single_state = single_state.sort_values("date")
    single_state['month'] = pd.DatetimeIndex(single_state['date']).month
    single_state['day'] = pd.DatetimeIndex(single_state['date']).day
    for month in months:
        #print(state)
        try:
            single_month = single_state.loc[single_state["month"] == month]
            single_month = single_month.reset_index()
            first_row = single_month.iloc[0]
            last_row = single_month.iloc[-1]
            month_pos = int(last_row["positive"]) - int(first_row["positive"])
            month_tot = int(last_row["posNeg"]) - int(first_row["posNeg"])
            positive_rate = month_pos / month_tot
            positive_rate = "{:.2%}".format(positive_rate)
            positive_rates.append(positive_rate)
            state_list.append(state)
            month_list.append(month)
            
            
        except IndexError:
            #some states had no cases in the first 2 months
            print("no data for this month")
        except ValueError:
            #New Jersey had had values of NaN causing a value error
            print("value issue")
        except ZeroDivisionError:
            #Washington first month total comes out to 0 causing a division by zero error
            print("month total = 0")

no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
value issue
value issue
value issue
no data for this month
no data for this month
no data for this month
no data for this month
no data for this month
no data for this mont

In [20]:
#Dataframe created from above loop
pos_rates_df = pd.DataFrame({
    "State": state_list,
    "Positive Rate": positive_rates,
    "Month": month_list
})
pos_rates_df.head(25)

,State,Positive Rate,Month
0,AK,3.21%,3
1,AK,1.53%,4
2,AK,0.22%,5
3,AK,0.82%,6
4,AK,2.27%,7
5,AK,2.09%,8
6,AL,13.48%,3
7,AL,7.48%,4
8,AL,8.55%,5
9,AL,10.75%,6


In [ ]:
#States total population by summing all counties population
state_pop = []
states = county_population_file["State"].unique()

for state in states:
    state_df = county_population_file.loc[county_population_file["State"] == state]
    state_pop.append(state_df["population"].sum())
    
population_df = pd.DataFrame({
    "state": states,
    "population": state_pop
})